In [1]:
import numpy as np
from tensorflow import keras

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
from classifier_net import ClassifierNet

test_classifier = ClassifierNet(input_shape=input_shape, num_classes=num_classes)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
test_classifier.train(x_train, y_train, epochs=1)

Train on 54000 samples, validate on 6000 samples
54000/54000 [==============================] - 2s 38us/sample - loss: 1.7082 - acc: 0.3544 - val_loss: 1.3394 - val_acc: 0.4763


In [4]:
test_weights = test_classifier.get_weights()
print(test_weights[test_weights != 0.0])
#print(test_classifier.get_training_data()['loss'])

flatten
dense
[-0.07884736 -0.05742773  0.00931647 ...  0.26699978  0.
  0.        ]
dense_1
[-0.13488224  0.50119114 -0.9888394   0.          0.         -0.75691766
 -1.0515034  -0.2670893   0.          0.          0.32542428  0.5256375
  0.7960033   0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.14138228  0.1822833   0.16337319  0.          0.        ]
dense_2
[-0.1185361   1.306247   -0.4157006   0.          0.         -0.25129402
  0.952854    0.6146657   0.          0.          1.039692   -0.02880218
 -0.41010815  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.24509627 -0.02637718  0.47454077  0.          0.        ]
dense_3
[ 0.5076386  -0.04711919  0.14946395 -0.14371671 -0.5855098   0.19434243
 -0.34977672  0.70090955  0.03038925  0.8344366  -0.9906259   0.59248406
 -0.59647     0.51863223  0.01445855  0.      

In [5]:
score = test_classifier.evaluate(x_test, y_test)

print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.3874889873504639
Test accuracy: 0.4602
